# 层和块

 块（block）可以描述单个层、由多个层组成的组件或整个模型本身。 使用块进行抽象的一个好处是可以将一些块组合成更大的组件， 这一过程通常是递归的，

In [24]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2,20)
net(X)

tensor([[-0.1976, -0.1135, -0.0866, -0.0454,  0.1366,  0.1997,  0.0072,  0.2198,
         -0.0887, -0.0399],
        [-0.0962, -0.2462, -0.0411, -0.0836, -0.0067,  0.2790,  0.0945,  0.1849,
          0.0103, -0.0280]], grad_fn=<AddmmBackward0>)

### 1. 自定义块

在下面的代码片段中，我们从零开始编写一个块。 它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。 注意，下面的MLP类继承了表示块的类。 我们的实现只需要提供我们自己的构造函数（Python中的__init__函数）和前向传播函数。

In [25]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256) 
        self.out = nn.Linear(256, 10)

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))


In [26]:
net = MLP()
net(X)

tensor([[-0.2607, -0.1793,  0.2608,  0.2187, -0.1825,  0.1298, -0.0344,  0.2432,
         -0.0575,  0.0900],
        [-0.1959, -0.0885,  0.2588,  0.1627, -0.0738,  0.1083, -0.0865, -0.0289,
          0.1315,  0.0375]], grad_fn=<AddmmBackward0>)

### 2. 顺序块

In [27]:
class MySequential(nn.Module):
    def __init__(self, *arg):        
        super().__init__()
        for idx, module in enumerate(arg):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。module的类型是OrderedDict
            self._modules[str(idx)] = module
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

__init__函数将每个模块逐个添加到有序字典_modules中。 你可能会好奇为什么每个Module都有一个_modules属性？ 以及为什么我们使用它而不是自己定义一个Python列表？ 简而言之，_modules的主要优点是： 在模块的参数初始化过程中， 系统知道在_modules字典中查找需要初始化参数的子块。

In [28]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1067, -0.1153, -0.1821, -0.1461,  0.0233,  0.0559, -0.1749,  0.0456,
          0.1170,  0.1830],
        [-0.0098, -0.0884, -0.1812, -0.1556,  0.1049, -0.0359, -0.1770,  0.2031,
          0.0251,  0.1600]], grad_fn=<AddmmBackward0>)

### 3. 在前向传播函数中执行代码

In [29]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [30]:
net = FixedHiddenMLP()
net(X)

tensor(0.0172, grad_fn=<SumBackward0>)

In [31]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.3260, grad_fn=<SumBackward0>)

### 4. 小结

- 一个块可以由许多层组成；一个块可以由许多块组成。

- 块可以包含代码。

- 块负责大量的内部处理，包括参数初始化和反向传播。

- 层和块的顺序连接由Sequential块处理。